In [1]:
from math import *
import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point, Polygon, MultiPolygon,MultiLineString
from bokeh.sampledata.us_counties import data as counties
import westernintnet


In [4]:
c = pd.DataFrame.from_dict(counties)

In [8]:
c.T.head()

detailed name  \
1 1  Autauga County, Alabama   
  3  Baldwin County, Alabama   
  5  Barbour County, Alabama   
  7     Bibb County, Alabama   
  9   Blount County, Alabama   

                                                  lats  \
1 1  [32.4757, 32.46599, 32.45054, 32.44245, 32.439...   
  3  [30.28557, 30.21934, 30.21771, 30.21183, 30.20...   
  5  [32.02221, 32.02066, 32.0135, 32.00249, 31.996...   
  7  [33.13143, 33.13086, 33.15133, 33.18184, 33.18...   
  9  [33.949, 33.95621, 33.9629, 33.97324, 33.99538...   

                                                  lons     name state  
1 1  [-86.41182, -86.41177, -86.41167, -86.41157, -...  Autauga    al  
  3  [-87.51203, -87.56704, -87.5741, -87.59954, -8...  Baldwin    al  
  5  [-85.04884, -85.05367, -85.05381, -85.06454, -...  Barbour    al  
  7  [-87.23637, -87.21582, -87.19914, -87.19907, -...     Bibb    al  
  9  [-86.8332, -86.81779, -86.79248, -86.7719, -86...   Blount    al

In [36]:
len(list(set(c.T['state'].values)))

56

In [37]:
np.unique(c.T['state'].values)

array(['ak', 'al', 'ar', 'as', 'az', 'ca', 'co', 'ct', 'dc', 'de', 'fl',
       'ga', 'gu', 'hi', 'ia', 'id', 'il', 'in', 'ks', 'ky', 'la', 'ma',
       'md', 'me', 'mi', 'mn', 'mo', 'mp', 'ms', 'mt', 'nc', 'nd', 'ne',
       'nh', 'nj', 'nm', 'nv', 'ny', 'oh', 'ok', 'or', 'pa', 'pr', 'ri',
       'sc', 'sd', 'tn', 'tx', 'ut', 'va', 'vi', 'vt', 'wa', 'wi', 'wv',
       'wy'], dtype=object)

In [10]:
ca_county_list

['San Bernardino',
 'San Diego',
 'Shasta',
 'San Benito',
 'Fresno',
 'Contra Costa',
 'San Mateo',
 'Glenn',
 'Del Norte',
 'Sierra',
 'Kings',
 'Tuolumne',
 'Alpine',
 'Trinity',
 'Monterey',
 'Lassen',
 'Marin',
 'Amador',
 'Sacramento',
 'Los Angeles',
 'Lake',
 'Imperial',
 'Modoc',
 'Tulare',
 'Kern',
 'San Joaquin',
 'San Luis Obispo',
 'Orange',
 'Santa Barbara',
 'Stanislaus',
 'Sonoma',
 'El Dorado',
 'Nevada',
 'Merced',
 'Yolo',
 'Santa Cruz',
 'Mariposa',
 'Sutter',
 'Mono',
 'Plumas',
 'Placer',
 'Ventura',
 'Humboldt',
 'Yuba',
 'Butte',
 'Alameda',
 'Madera',
 'Calaveras',
 'Tehama',
 'Solano',
 'Colusa',
 'Santa Clara',
 'Mendocino',
 'Siskiyou',
 'Riverside',
 'Inyo',
 'San Francisco',
 'Napa']

In [ ]:

#? CA has to be split in 5 parts, but we have that info already


#### What is 'El Paso'? Is that counties? Look in EIA-861 file? 
EIA-861 file Service_territory_2016.xlsx says EPE services the ff counties:
* Culberson
* El Paso
* Hudspeth

Eyeballing the EPE BA map against the Texas Counties map suggests it's these 3 counties, not just El Paso. (Which also means I underestimated the EPE demand since I only took the El Paso population. But we want to stay consistent, so use El Paso county for now, since the demand counts are just from El Paso, then update later with the populations for the additional counties and rerun the simulation.


In [21]:
#wecc_regions same as demand_profile headers
wecc_regions = ['WA', 'OR', 'CAnorth', 'CABayArea', 'CAcentral', 'CAsw',
       'CAse', 'NV', 'AZ', 'UT', 'NM', 'CO', 'WY', 'ID', 'MT', 'ElPaso']  

wecc_states = ['wa','or','ca','nv','az','ut','nm','co','wy','id','mt']

CA_regions = ['CAnorth', 'CABayArea', 'CAcentral', 'CAsw','CAse']

Northern_California = ['Butte','Colusa','Del Norte','El Dorado','Glenn','Humboldt',
                       'Lake','Lassen','Mendocino','Modoc','Nevada','Placer','Plumas',
                       'Sacramento','Shasta','Sierra','Siskiyou','Sutter','Tehama',
                       'Trinity','Yolo','Yuba']
Bay_Area = ['Alameda','Contra Costa','Marin','Napa','San Francisco','San Mateo',
            'Santa Clara','Santa Cruz','Solano','Sonoma']
Central_California = ['Alpine','Amador','Calaveras','Fresno','Inyo','Kings','Madera',
                      'Mariposa','Merced','Mono','Monterey','San Benito','San Joaquin',
                      'Stanislaus','Tulare','Tuolumne']
Southeast_California = ['Imperial','Orange','Riverside','San Bernardino','San Diego']
Southwest_California = ['Kern','Los Angeles','San Luis Obispo','Santa Barbara','Ventura']

CA = [Northern_California, Bay_Area, Central_California, Southeast_California, Southwest_California]
CAnames = ['Northern_California', 'Bay_Area', 'Central_California', 'Southeast_California', 'Southwest_California']
CA_all = Northern_California + Bay_Area + Central_California + Southeast_California +Southwest_California
CA_union = list(set().union(Northern_California, Bay_Area, Central_California, Southeast_California, Southwest_California))
shortCAnames = ['CAnorth','CABayArea','CAcentral','CAsw','CAse']



In [50]:
county_list = {}
for state in [r for r in wecc_regions if ((r[0:2] != 'CA') & (r != 'ElPaso'))]:
    county_list[state] = list({county['name'] for code, county in counties.items() if county["state"] == state.lower()})
    
for ca in zip(CA_regions,CA):
    county_list[ca[0]] = ca[1]
    
county_list['ElPaso'] = ['El Paso']
#Fix to
#county_list['ElPaso'] = ['El Paso', 'Culberson', 'Hudspeth']

In [51]:
county_list

{'WA': ['Ferry',
  'Walla Walla',
  'Jefferson',
  'Whitman',
  'Lincoln',
  'Pend Oreille',
  'Skagit',
  'Island',
  'Yakima',
  'Columbia',
  'Klickitat',
  'Okanogan',
  'Snohomish',
  'Spokane',
  'Cowlitz',
  'King',
  'Benton',
  'Adams',
  'Asotin',
  'Franklin',
  'Garfield',
  'Clallam',
  'Clark',
  'Whatcom',
  'Stevens',
  'Pierce',
  'Thurston',
  'Lewis',
  'Mason',
  'Kitsap',
  'Grays Harbor',
  'Pacific',
  'San Juan',
  'Skamania',
  'Kittitas',
  'Grant',
  'Wahkiakum',
  'Douglas',
  'Chelan'],
 'OR': ['Wasco',
  'Curry',
  'Josephine',
  'Wallowa',
  'Umatilla',
  'Clatsop',
  'Jefferson',
  'Harney',
  'Gilliam',
  'Lincoln',
  'Baker',
  'Klamath',
  'Polk',
  'Columbia',
  'Malheur',
  'Tillamook',
  'Deschutes',
  'Hood River',
  'Union',
  'Benton',
  'Lake',
  'Washington',
  'Yamhill',
  'Linn',
  'Multnomah',
  'Wheeler',
  'Coos',
  'Grant',
  'Jackson',
  'Lane',
  'Sherman',
  'Crook',
  'Morrow',
  'Clackamas',
  'Marion',
  'Douglas'],
 'NV': ['Storey

In [55]:
df0 = pd.DataFrame.from_dict(county_list, orient = 'index')

In [56]:
df1=df.T
df1 = df1[wecc_regions]

In [58]:
citiesdir = 'U:\\src\\PostREISE\\postreise\\analyze\\TransmissionLineAnalysis\\'
df1.to_csv(citiesdir + 'weccZones_2016.csv', index=False)

In [27]:
[y for y in wecc_regions if y[0:2] != 'CA']

['WA', 'OR', 'NV', 'AZ', 'UT', 'NM', 'CO', 'WY', 'ID', 'MT', 'ElPaso']

In [12]:
county_list

{'wa': ['Ferry',
  'Walla Walla',
  'Jefferson',
  'Whitman',
  'Lincoln',
  'Pend Oreille',
  'Skagit',
  'Island',
  'Yakima',
  'Columbia',
  'Klickitat',
  'Okanogan',
  'Snohomish',
  'Spokane',
  'Cowlitz',
  'King',
  'Benton',
  'Adams',
  'Asotin',
  'Franklin',
  'Garfield',
  'Clallam',
  'Clark',
  'Whatcom',
  'Stevens',
  'Pierce',
  'Thurston',
  'Lewis',
  'Mason',
  'Kitsap',
  'Grays Harbor',
  'Pacific',
  'San Juan',
  'Skamania',
  'Kittitas',
  'Grant',
  'Wahkiakum',
  'Douglas',
  'Chelan'],
 'or': ['Wasco',
  'Curry',
  'Josephine',
  'Wallowa',
  'Umatilla',
  'Clatsop',
  'Jefferson',
  'Harney',
  'Gilliam',
  'Lincoln',
  'Baker',
  'Klamath',
  'Polk',
  'Columbia',
  'Malheur',
  'Tillamook',
  'Deschutes',
  'Hood River',
  'Union',
  'Benton',
  'Lake',
  'Washington',
  'Yamhill',
  'Linn',
  'Multnomah',
  'Wheeler',
  'Coos',
  'Grant',
  'Jackson',
  'Lane',
  'Sherman',
  'Crook',
  'Morrow',
  'Clackamas',
  'Marion',
  'Douglas'],
 'ca': ['San Be

In [13]:
county_list['ca']

['San Bernardino',
 'San Diego',
 'Shasta',
 'San Benito',
 'Fresno',
 'Contra Costa',
 'San Mateo',
 'Glenn',
 'Del Norte',
 'Sierra',
 'Kings',
 'Tuolumne',
 'Alpine',
 'Trinity',
 'Monterey',
 'Lassen',
 'Marin',
 'Amador',
 'Sacramento',
 'Los Angeles',
 'Lake',
 'Imperial',
 'Modoc',
 'Tulare',
 'Kern',
 'San Joaquin',
 'San Luis Obispo',
 'Orange',
 'Santa Barbara',
 'Stanislaus',
 'Sonoma',
 'El Dorado',
 'Nevada',
 'Merced',
 'Yolo',
 'Santa Cruz',
 'Mariposa',
 'Sutter',
 'Mono',
 'Plumas',
 'Placer',
 'Ventura',
 'Humboldt',
 'Yuba',
 'Butte',
 'Alameda',
 'Madera',
 'Calaveras',
 'Tehama',
 'Solano',
 'Colusa',
 'Santa Clara',
 'Mendocino',
 'Siskiyou',
 'Riverside',
 'Inyo',
 'San Francisco',
 'Napa']

In [14]:
len(county_list['ca'])

58

In [16]:
len(CA_union)

58

In [18]:
county_list.keys()

dict_keys(['wa', 'or', 'ca', 'nv', 'az', 'ut', 'nm', 'co', 'wy', 'id', 'mt'])